In [149]:
# import Libraries
import os

import langchain
import openai
from dotenv import load_dotenv
from langchain import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from pinecone import Pinecone


In [150]:
load_dotenv()

True

In [151]:
## LOAD DOCS FROM DIRECTORY
def read_doc(directory):
  file_loader = PyPDFDirectoryLoader(directory)
  documents = file_loader.load()
  return documents


In [152]:
pdf_dir = os.path.join("..", "pdfs")
doc=read_doc(pdf_dir)

In [153]:
# Convert into chunks
# https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunks=text_splitter.split_documents(docs)
    return chunks

In [154]:
chunked_documents = chunk_data(docs=doc)
len(chunked_documents)

7

In [157]:
## Embedding Technique of OpenAI
embeddings=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
vectors = embeddings.embed_query("How are you?")
len(vectors)

In [ ]:
## Vector Search DB In Pinecone
# pinecone.init(
#     api_key=os.getenv("PINECONE_API_KEY"),
#     environment=os.getenv("PINECONE_ENV")
# )

# pc = Pinecone(
#   api_key=os.environ.get("PINECONE_AP_KEY"),
#   environment=os.getenv("PINECONE_ENV"),
# )

index_name="docs"
pinecone = Pinecone.from_documents(
  chunked_documents, embeddings, index_name=index_name,
)


In [ ]:
## Cosine Similarity Retrieval Results from VectorDB
def retrieve_query(query, k=2):
  matching_results=pinecone.similarity_search(query, k=k)
  return matching_results

In [ ]:
llm=OpenAI(model_name="gpt-3.5-turbo") 
chain=load_qa_chain(llm,chain_type="stuff")

In [ ]:
## Search answers from VectorDB
def retrieve_answers(query):
  doc_search=retrieve_query(query)
  print(doc_search)
  response = chain.run(input_documents=doc_search, question=query)
  return response

In [ ]:
our_query = "How is the agriculture doing?"
answer = retrieve_answers(our_query)
print(answer)